In [1]:
from configs import vanilla_vae
config = vanilla_vae.get_config()
config

dataset: Organoid
device: cuda
epochs: 10000
hidden_dims:
- 32
- 32
- 32
in_features: 41
kld_weight: 0.0025
latent_dim: 2
learning_rate: 0.005
model: VAE
save_loss_every_n_epochs: 10
seed: 12345
weight_decay: 0.0

In [6]:
from models import VanillaVAE
config.hidden_dims = [32,16,8]
model = VanillaVAE(config).to(config.device)

In [7]:
model

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=41, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=16, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_mu): Linear(in_features=8, out_features=2, bias=True)
  (fc_var): Linear(in_features=8, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, trac

In [4]:
from datasets import OrganoidDataset
data = OrganoidDataset()

X_train,y_train = data.train
X_val,y_val = data.val

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [9]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )
optimizer.zero_grad()
outputs = model.forward(X_train)
loss = model.loss_function(*outputs, config=config)['loss']
loss.backward()
optimizer.step()

In [15]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/vanilla_vae.py --config.epochs=10000')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 10000/10000 [14:01<00:00, 11.88it/s]


Finished Training


0

In [1]:
from configs import wae_mmd
config = wae_mmd.get_config()
config

batch_size: 4096
dataset: Organoid
device: cuda
epochs: 10000
hidden_dims:
- 32
- 32
- 32
in_features: 41
kernel_type: imq
latent_dim: 2
latent_var: 1
learning_rate: 0.005
model: WAE_MMD
reg_weight: 1
save_loss_every_n_epochs: 10
seed: 12345
weight_decay: 0.0

In [2]:
from models import WAE_MMD
config.hidden_dims = [32,16,8]
model = WAE_MMD(config).to(config.device)
model

WAE_MMD(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=41, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=16, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_z): Linear(in_features=8, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
 

In [3]:
from datasets import OrganoidDataset

data = OrganoidDataset()

X_train, y_train = data.train
X_val, y_val = data.val

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
import torch

X_train_batches = torch.split(X_train,split_size_or_sections=config.batch_size)
X_val_batches = torch.split(X_train,split_size_or_sections=config.batch_size)

In [5]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )
optimizer.zero_grad()
outputs = model.forward(X_train_batches[0])
loss = model.loss_function(*outputs, config=config)['loss']
loss.backward()
optimizer.step()

In [21]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/wae_mmd.py --config.epochs=200')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 200/200 [17:26<00:00,  5.23s/it]


Finished Training


0

In [29]:
import os
for lr in [0.05,0.01,0.005,0.001]:
    os.system(f'python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
              f'--config=configs/vanilla_vae.py '
              f'--config.output_dir=logs/VanillaVAE/grid_example '
              f'--config.hidden_dims="(41,41,41,41)" '
              f'--config.learning_rate={lr} ')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 10000/10000 [25:34<00:00,  6.52it/s]


Finished Training


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


...

Finished Training


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 10000/10000 [24:32<00:00,  6.79it/s]


Finished Training
